In [76]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [ ]:

st_accept = "text/html" # говорим веб-серверу,
                        # что хотим получить html
# имитируем подключение через браузер Mozilla на macOS
st_useragent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"
# формируем хеш заголовков
headers = {
   "Accept": st_accept,
   "User-Agent": st_useragent
}



def get_links_from_map():
   # отправляем запрос с заголовками по нужному адресу
   req = requests.get("https://tabiturient.ru/map/", headers)
   # считываем текст HTML-документа
   src = req.text

   soup = BeautifulSoup(src, 'html.parser')
   # Извлекаем все теги <a> с атрибутом href
   links = [link.get('href') for link in soup.find_all('a') if link.get('href')]
   with open('links.txt', 'w') as f:
      for link in links:
         f.write(f"{link}\n")

In [77]:
data = pd.read_csv('links.txt')

In [78]:
data.columns =["url"]

In [79]:
data

,url
0,https://tabiturient.ru
1,https://tabiturient.ru/np
2,https://tabiturient.ru/calculator
3,https://tabiturient.ru/calculator
4,https://tabiturient.ru/calculator/pay
...,...
77316,https://tabiturient.ru/feedback
77317,https://tabiturient.ru/admin2/ca.php
77318,https://tabiturient.ru/adv
77319,https://tabiturient.ru/vuzam2


In [80]:
pattern = r"https://tabiturient\.ru/vuzu/[A-Za-z0-9_.-]+/proxodnoi$"

# Проверяем, содержит ли хотя бы одна ячейка строки нужный шаблон
mask = data.apply(lambda row: row.str.contains(pattern, regex=True, na=False).any(), axis=1)
data_vizu = data[mask]

In [81]:
data_vizu

,url
41,https://tabiturient.ru/vuzu/agatu/proxodnoi
68,https://tabiturient.ru/vuzu/bigpibiysk/proxodnoi
102,https://tabiturient.ru/vuzu/altgaki/proxodnoi
134,https://tabiturient.ru/vuzu/agiki/proxodnoi
156,https://tabiturient.ru/vuzu/asmu/proxodnoi
...,...
25811,https://tabiturient.ru/vuzu/theatrinsyar/proxo...
25819,https://tabiturient.ru/vuzu/ystu/proxodnoi
25875,https://tabiturient.ru/vuzu/uniyarac/proxodnoi
25916,https://tabiturient.ru/vuzu/yarpgups/proxodnoi


In [90]:
data_vizu.to_csv('vizu_lins.csv', index=False)

In [83]:
pattern = r"https://tabiturient\.ru/vuzu/.*/proxodnoi$"

# Проверяем, содержит ли хотя бы одна ячейка строки нужный шаблон
mask = data.apply(lambda row: row.str.contains(pattern, regex=True, na=False).any(), axis=1)
data_vizu_dup = data[mask]

In [84]:
data_vizu_dup

,url
41,https://tabiturient.ru/vuzu/agatu/proxodnoi
68,https://tabiturient.ru/vuzu/bigpibiysk/proxodnoi
102,https://tabiturient.ru/vuzu/altgaki/proxodnoi
134,https://tabiturient.ru/vuzu/agiki/proxodnoi
156,https://tabiturient.ru/vuzu/asmu/proxodnoi
...,...
25811,https://tabiturient.ru/vuzu/theatrinsyar/proxo...
25819,https://tabiturient.ru/vuzu/ystu/proxodnoi
25875,https://tabiturient.ru/vuzu/uniyarac/proxodnoi
25916,https://tabiturient.ru/vuzu/yarpgups/proxodnoi


In [85]:
exclusive_df1 = data_vizu_dup[~data_vizu_dup['url'].isin(data_vizu['url'])]

In [86]:
exclusive_df1

,url
452,https://tabiturient.ru/vuzu/ачгаа/proxodnoi
1929,https://tabiturient.ru/vuzu/билгу/proxodnoi
5049,https://tabiturient.ru/vuzu/дрти/proxodnoi


In [91]:
pattern = r"https://tabiturient\.ru/vuzu/[A-Za-z0-9_.-]+/obsh"

# Проверяем, содержит ли хотя бы одна ячейка строки нужный шаблон
mask = data.apply(lambda row: row.str.contains(pattern, regex=True, na=False).any(), axis=1)
data_vizu_obsh = data[mask]

In [ ]:
data_vizu_obsh

,url
65,https://tabiturient.ru/vuzu/agatu/obsh
99,https://tabiturient.ru/vuzu/bigpibiysk/obsh
131,https://tabiturient.ru/vuzu/altgaki/obsh
153,https://tabiturient.ru/vuzu/agiki/obsh
164,https://tabiturient.ru/vuzu/asmu/obsh
...,...
25816,https://tabiturient.ru/vuzu/theatrinsyar/obsh
25872,https://tabiturient.ru/vuzu/ystu/obsh
25913,https://tabiturient.ru/vuzu/uniyarac/obsh
25923,https://tabiturient.ru/vuzu/yarpgups/obsh


```
{
	vuz: {
		long_name: string // полное название,
		short_name: string // аббревиатура вуза
		geolocation: string // регион
		is_goverment: boolean // True если гос вуз
		rating: string // рейтинг (пока буквами)
		logo: string // ссылка на фото
		website: string // ссылка на сайт вуза
	},
	programs: [
		{
			!direction: string // направление
			!profile: string // профиль
			!program_code: string // код программы
			!vuz: string // аббревиатура вуза
			!faculty: string // факультет
			!exams: [[string]] // набор экзаменов
			!scores: [[score, form]] // баллы
			!education_form: string // форма обучения
			?free_places: int // бюджетных мест
			?average_score: float // средни проходной балл
			?olympic: int // по олимпиаде поступило
			?price: int // стоимость обучения
		}
	]

}
```